# R&S ZNB/ZNBT VNA - Optimize test program

- **Author:** Juan del Pino Mena
- **Version:** v01
- **Date:** 2024-02-19


---

# Introduction

This program performs a simple single optimization process

## Requirements

- `numpy`: For vector manipulation
- `matplotlib`: For plotting data
- `socketscpi`: For communication with SCPI commands over sockets.
    - docs: https://socketscpi.readthedocs.io/en/latest/index.html
    - original repo: https://github.com/morgan-at-keysight/socketscpi
    - backup fork: https://github.com/dpmj/socketscpi

Execute this notebook on a virtual environment for safety: https://medium.com/@royce963/setting-up-jupyterlab-and-a-virtual-environment-c79002e0e5f7


## VNA network interface configuration

1. Connect the function generator and your PC to the same local network. **They even can be connected directly via an Ethernet cable.**
2. Make sure you are connected to the same LAN as the VNA, and that they have compatible IP addresses. To check your IP, type `ipconfig` (windows), or `ifconfig` // `ip addr` (linux).
3. For convenience, this guide sets a fixed IP to the VNA.
4. Check if the device is accessible: Do a ping to the device: `ping 10.10.0.152` (windows & linux).


## Fixed IP setup on the VNA

**By default, the R&S VNA has address `10.10.0.152`, with mask `/8`.** Changing the IP is possible but requires an admin password. **In this document, this IP is not changed** If needed, follow these steps:

1. Press the windows key
2. Control panel >> Network and sharing center >> Change adapter settings >> local connection (right click) >> Properties
3. Connection properties >> Internet Protocol Version 4 (TCP/IPv4) >> Properties
4. Enter the IP configuration
5. The port for programming the VNA shall be always `5025`

## Fixed IP setup on your PC

- On Linux:
    `sudo ip addr add 10.10.0.151/8 dev enp3s0` (CHANGE `enp3s0` ACCORDING TO THE NAME OF YOUR ETHERNET INTERFACE)
- On Windows: similar steps as in the VNA.

## Calibration

Ensure the VNA is properly calibrated before measuring. There shall be a calibration file already available and suitable on the VNA. Go to CAL >> USE CAL >> CAL MANAGER. In the Pool (right window), select "Reconf-Filter-2-6G-201". Contains the calibration data for a 2-to-6 GHz bandwidth with 201 points per trace using the Agilent 85052C calibration kit. Once selected, click "Apply" on the center of the screen.

## Known problems

When using the `socketscpi` package, this error may appear on screen:

    Remote error -113: Undefined header;
    syst:err:verbose 1

This is due an internal command `socketscpi` sends to the VNA trying to explain other errors. This is not important and can be ignored. To avoid this, set `verboseErrCheck=False` when calling `socketscpi.SocketInstrument()`. See below.

---

# Program & measurement config

In [2]:
# Import dependencies. Install any missing dependencies with pip/conda inside the virtual environment

import socketscpi
import numpy as np
from scipy import optimize
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'socketscpi'

In [12]:
# Parameters - global variables

HOST = "10.10.0.152"  # [str] Instrument IP address. Default: "10.10.0.152"
PORT = 5025  # [int] Instrument listening port. Default: 5025
TIMEOUT = 10  # [s] How many seconds to wait for a response. Default: 10

F_MIN = 2e9  # [Hz] Default: 2e9 (2 GHz)
F_MAX = 6e9  # [Hz] Default: 6e9 (6 GHz)
N_POINTS = 201  # Number of measurement points

FREQ = np.linspace(F_MIN, F_MAX, N_POINTS)  # Frequency vector, for plotting

# Functions

In [13]:
def vna_setup():
    """
    Performs a reset and setups the VNA with default known parameters. 
    """
    
    vna = socketscpi.SocketInstrument(ipAddress=HOST, 
                                      port=PORT, 
                                      timeout=TIMEOUT, 
                                      verboseErrCheck = False)
    
    print(vna.instId)  # Identifies the instrument
    
    # ####################################################################################
    # Reset

    vna.write("*RST")  # Reset
    vna.write("INIT:CONT:ALL OFF")  # Disables continuous mode even for new traces
    vna.write("CALC:PAR:DEL:ALL")  # Delete all traces, blanks screen
    vna.write("*WAI")  # Waits until completed before proceeding with next command

    # ####################################################################################
    # Establishes start, stop and number of points
    
    vna.write('SENS:FREQ:STAR ' + str(F_MIN))  # Set lower frequency bound
    vna.write('SENS:FREQ:STOP ' + str(F_MAX))  # Set upper frequency bound
    vna.write('SENS:SWE:POIN ' + str(N_POINTS))  # Set number of points per trace
    vna.write("*WAI")  # Waits until completed before proceeding with next command

    # ####################################################################################
    # Creates new traces and measurements. Pag 864
    # Channel 1 (CALC1, number omitted). Measures magnitude in dB and phase in degrees
    
    vna.write("CALC:PAR:SDEF 'Trc1_mlog', 'S11'")
    vna.write("CALC:PAR:SEL 'Trc1_mlog'")
    vna.write("CALC:FORM MLOG")  # MLOG: Magnitude,dB. Pag 807
    
    vna.write("CALC:PAR:SDEF 'Trc1_phas', 'S11'")
    vna.write("CALC:PAR:SEL 'Trc1_phas'")
    vna.write("CALC:FORM PHAS")  # PHAS: Phase,deg. Pag 807
    
    vna.write("CALC:PAR:SDEF 'Trc2_mlog', 'S12'")
    vna.write("CALC:PAR:SEL 'Trc2_mlog'")
    vna.write("CALC:FORM MLOG")
    
    vna.write("CALC:PAR:SDEF 'Trc2_phas', 'S12'")
    vna.write("CALC:PAR:SEL 'Trc2_phas'")
    vna.write("CALC:FORM PHAS")
    
    vna.write("CALC:PAR:SDEF 'Trc3_mlog', 'S21'")
    vna.write("CALC:PAR:SEL 'Trc3_mlog'")
    vna.write("CALC:FORM MLOG")
    
    vna.write("CALC:PAR:SDEF 'Trc3_phas', 'S21'")
    vna.write("CALC:PAR:SEL 'Trc3_phas'")
    vna.write("CALC:FORM PHAS")
    
    vna.write("CALC:PAR:SDEF 'Trc4_mlog', 'S22'")
    vna.write("CALC:PAR:SEL 'Trc4_mlog'")
    vna.write("CALC:FORM MLOG")
    
    vna.write("CALC:PAR:SDEF 'Trc4_phas', 'S22'")
    vna.write("CALC:PAR:SEL 'Trc4_phas'")
    vna.write("CALC:FORM PHAS")
    
    vna.write("*WAI")  # Waits until completed before proceeding with next command
    
    # ####################################################################################
    # Display traces on the screen. Single sweep (only one measurement)
    
    vna.write("DISP:WIND1:STAT ON")  # Turn on window 1
    
    vna.write("DISP:WIND1:TRAC1:FEED 'Trc1_mlog'")  # Add traces
    vna.write("DISP:WIND1:TRAC2:FEED 'Trc2_mlog'")
    vna.write("DISP:WIND1:TRAC3:FEED 'Trc3_mlog'")
    vna.write("DISP:WIND1:TRAC4:FEED 'Trc4_mlog'")
    
    vna.write("DISP:WIND1:TRAC5:FEED 'Trc1_phas'")
    vna.write("DISP:WIND1:TRAC6:FEED 'Trc2_phas'")
    vna.write("DISP:WIND1:TRAC7:FEED 'Trc3_phas'")
    vna.write("DISP:WIND1:TRAC8:FEED 'Trc4_phas'")
    
    vna.write("INIT:IMM; *WAI")  # Perform a single sweep and wait until completed

    # ####################################################################################
    # Set data format - Swapped byte order, float32.
    
    vna.write('format:border swap')  # Swapped byte order
    vna.write('format real,32')  # 32-bit precision float
    vna.write("*WAI")  # Waits until completed before proceeding with next command

    # ####################################################################################
    # Check errors. Close socket.
    
    vna.err_check()
    vna.close()

In [14]:
def vna_measure_once():
    """
    Performs a sweep and measure all data. Returns the measurement as a matrix.
    """
    
    vna = socketscpi.SocketInstrument(ipAddress=HOST, 
                                      port=PORT, 
                                      timeout=TIMEOUT, 
                                      verboseErrCheck = False)
    
    vna.write("INIT:IMM; *WAI")  # Perform a single sweep and wait until completed
    
    vna.write('format:border swap')  # Swapped byte order
    vna.write('format real,32')  # 32-bit precision float
    vna.write("*WAI")  # Waits until completed before proceeding with next command
    
    data = vna.query_binary_values('CALC:DATA:ALL? FDAT', datatype='f')
    
    vna.err_check()
    vna.close()

    meas = data.reshape(8, N_POINTS)  # Reshape: 8 columns, of N_POINTS rows.
    
    return meas

In [15]:
def plot_measurement(measurement):
    """
    Plots data (in the expected 8-column, of N_POINTS-row format.)
    """

    fig, (ax1, ax2) = plt.subplots(2, figsize=[12, 10])
    
    fig.suptitle("Data retrieved from VNA - Magnitude and phase")
    ax1.set_title('Data - Magnitude')
    # ax1.set_xlabel('Freq (Hz)')
    ax1.set_ylabel('Magnitude (dB)')
    ax2.set_title('Data - Phase')
    ax2.set_xlabel('Freq (Hz)')
    ax2.set_ylabel('Phase (º)')
    
    ax1.plot(FREQ, measurement[0][:], label="Trc1,S11,mlog")
    ax1.plot(FREQ, measurement[2][:], label="Trc2,S21,mlog")
    ax1.plot(FREQ, measurement[4][:], label="Trc3,S12,mlog")
    ax1.plot(FREQ, measurement[6][:], label="Trc4,S22,mlog")
    
    ax1.grid(True, which='major', color='#DDDDDD', linestyle='-', linewidth=0.8)
    ax1.grid(True, which='minor', color='#DDDDDD', linestyle=':', linewidth=0.8)
    ax1.minorticks_on()
    ax1.legend(loc="center right")
    
    plt.plot(FREQ, measurement[1][:], label="Trc1,S11,phas")
    plt.plot(FREQ, measurement[3][:], label="Trc2,S21,phas")
    plt.plot(FREQ, measurement[5][:], label="Trc3,S12,phas")
    plt.plot(FREQ, measurement[7][:], label="Trc4,S22,phas")
    
    ax2.grid(True, which='major', color='#DDDDDD', linestyle='-', linewidth=0.8)
    ax2.grid(True, which='minor', color='#DDDDDD', linestyle=':', linewidth=0.8)
    ax2.minorticks_on()
    ax2.legend(loc="center right")
        
    plt.show()

In [16]:
vna_setup()

TimeoutError: timed out

In [ ]:
meas = vna_measure_once()

In [9]:
plot_measurement(meas)

NameError: name 'meas' is not defined

# Optimization

## ERR function

The goal is to minimize an objective function, also called error function, which defines how much is the difference between the objectives and the response of the system. The optimization algorithm will simply search for the minimum of this function. 

Variables which act upon the response function are the DAC voltage settings.

In [10]:
import numpy as np

In [11]:
# Parameters - global variables
# Here re-declared for convenience

HOST = "10.10.0.152"  # [str] Instrument IP address. Default: "10.10.0.152"
PORT = 5025  # [int] Instrument listening port. Default: 5025
TIMEOUT = 10  # [s] How many seconds to wait for a response. Default: 10

F_MIN = 2e9  # [Hz] Default: 2e9 (2 GHz)
F_MAX = 6e9  # [Hz] Default: 6e9 (6 GHz)
N_POINTS = 201  # Number of measurement points

FREQ = np.linspace(F_MIN, F_MAX, N_POINTS)  # Frequency vector, for plotting

In [12]:
def addwindow(sparam, orientation, value, flow, fhigh, weight):
    """
    Defines an optimization window
    :param sparam: S-parameter to optimize: S11, S12, S21, S22. String.
    :param orientation: orientation: greater than, less than ('<','>')
    :param value: value to compare to in the opt, in dB: e.g.: S11 < -20 (dB)
    :param freq_lim_low: Lower frequency limit of the window, in GHz
    :param freq_lim_high: Upper frequency limit of the window, in GHz
    :param weight: Weight in the optimization algorithm. Scalar.
    :return a standardized dict with the above parameters
    """
    
    return {'sparam': sparam,  # parameter to optimize
            'orientation': orientation,  # orientation: greater than, less than ('<','>')
            'value': value,  # value to compare to in the opt.: e.g.: S11 < -20 (dB)
            'flow': freq_lim_low,  # Lower frequency limit, in GHz
            'fhigh': freq_lim_high,  # Upper frequency limit, in GHz
            'weight': weight}  # Weight in the optimization algorithm

In [13]:
def checkmasks(masks):
    """
    Checks mask in a masks list
    :param masks: optimization mask (list of dicts, format: see addwindow() func.)
    :param mat: s2p data matrix,
    """

    valid_sparam = ("S11", "S12", "S21", "S22")
    valid_orientation = ("<", ">")

    for mask in masks:
        
        flow = mask['flow']
        fhigh = mask['fhigh']
        sparam = mask['sparam']
        orientation = mask['orientation']
        
        if flow < F_MIN:
            raise Exception(f"flow cannnot be lower than F_MIN.\nMask: {mask}")
        if fhigh > F_MAX:
            raise Exception(f"fhigh cannnot be greater than F_MAX.\nMask: {mask}")
        if flow > fhigh:
            raise Exception(f"flow cannnot be greater than fhigh.\nMask: {mask}")
        if sparam not in valid_sparam:
            raise Exception(f"Unvalid S-param. Allowed: {valid_sparam}\nMask: {mask}")
        if orientation not in valid_orientation:
            raise Exception(f"Unvalid orientation. Allowed: {valid_orientation}\nMask: {mask}")

In [14]:
def evalerror(mat, masks):
    """
    Evaluates the error in the optimization algorithm
    :param mat: 2-port S-parameter matrix
    :param masks: optimization mask (list of dicts, format: see addwindow() func.)

    mat format:
    Npoints rows
    9 cols: f[Hz], s11(mag,pha)[dB], s21 (mag,pha)[dB], s12(mag,pha)[dB], s22(mag,pha)[dB]
    """

    # indicates the column index in the s2p matrix where the mangitude of each s-param is. 
    sparam_mag_col = {"S11": 1,
                      "S21": 3,
                      "S12": 5,
                      "S22": 7}
    
    checkmasks2(masks)  # Check masks. This should be done only once, but whatever
    
    # check matrix dimensions
    valid_shape = (N_POINTS, 9)
    shape = np.shape(mat)
    if (shape != valid_shape):  # check number of rows and cols
        raise Exception(f"Unexpected matrix shape: {shape}. Expected: {valid_shape}\n")
    
    error = 0  # total error, added in every iteration of mask check
    
    # Calculate the error
    for mask in masks:
        mask_error = 0  # per-mask error. 
    
        sparam = mask['sparam']
        value = mask['value']
        weight = mask['weight']
        orientation = mask['orientation']
        flow = mask['flow'] * 1e9
        fhigh = mask['fhigh'] * 1e9
    
        # Search the low and high frequency indexes that define the window inside the 
        # s2p-matrix, that we will use to calculate the error. If it doesn't match, it'll
        # use the most restrictive case (immediately lower or higher)
    
        # Find the index of the first value that is lower/greater or equal than the 
        # objective frequency. In the argmax() search, the FREQ array is flipped to ensure
        # that the found index does not narrow the defined window.
        
        flow_index = (len(FREQ) - 1) - np.argmax(FREQ[::-1] <= flow)
        fhigh_index = np.argmax(FREQ >= fhigh)

        # extracts sub-matrix of relevant s-param values from mat
        s_param_values = mat[flow_index:fhigh_index + 1, sparam_mag_col[sparam]]
        diff = value - s_param_values  # difference vector (how far from value?)
    
        if orientation == '>':  # 'greater than' the 'value'
            mask_error = weight * np.sum(diff[diff > 0])  # only sum them if diff > 0
        
        else:  # 'smaller than' the 'value'
            mask_error = weight * np.abs(np.sum(diff[diff < 0]))  # only sum them if diff < 0

        error += mask_error

    return error


In [2]:

shape = np.shape(mat)

NameError: name 'np' is not defined

# Optimization with scipy

Unconstrained minimization of multivariate scalar functions (minimize)

Examples from: https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [4]:
from scipy.optimize import minimize, Bounds, OptimizeResult
import numpy as np

In [5]:
x = np.arange(5)
y,z,u,v,w = x

In [15]:
def rosen(x, a, b, c):
    """The Rosenbrock function"""
    return sum((a*x[1:]-x[:-1]**a)**2.0 * b + c + (1-x[:-1])**2.0)

In [22]:
x1 = np.array([1.7, 2.0, 0.2, 1.1, 1.9])

bounds = ( (0,20), (0,20), (0,20), (0,20), (0,20) )

res1 = minimize(rosen, x1, args=(3, 210, 323), method='nelder-mead',
               options={'xatol': 1e-2, 'disp': True}, bounds=bounds)

Optimization terminated successfully.
         Current function value: 1293.259243
         Iterations: 131
         Function evaluations: 223


In [23]:
res1.x

array([1.57046864, 1.29111586, 0.71758374, 0.12342524, 0.        ])

In [24]:
historic = []

def opt_callback(intermediate_result):
    """
    A callback function which is called at the end of every iteration of the optimization. 
    Saves intermediate data in an historic
    :param intermediate_result: scipy.optimize.OptimizeResult object
    intermediate_result MUST be called this way, otherwise scipy wont pass the argument to
    the callback correctly
    """
    # Log into historic the
    historic.append((intermediate_result.fun, intermediate_result.x))  # function input variables

def rosen2(x, a, b):
    """
    The Rosenbrock function ((Modified for multivariable))
    """
    
    y,z,u,v,w = x
    result = a * b * 100.0*(y-z**2.0)**2.0 + w * (1-u*v)**2.0
    return result

In [32]:
x2 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
historic = []
res2 = minimize(rosen2, x2, method='nelder-mead', args=(3,2), callback=opt_callback, tol=1, options={'xatol': 1, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.792695
         Iterations: 22
         Function evaluations: 40


In [33]:
historic

[(np.float64(254.0878312581146), array([1.17 , 0.721, 0.824, 1.957, 1.236])),
 (np.float64(254.0878312581146), array([1.17 , 0.721, 0.824, 1.957, 1.236])),
 (np.float64(254.0878312581146), array([1.17 , 0.721, 0.824, 1.957, 1.236])),
 (np.float64(188.89431019248823),
  array([1.14712 , 0.765856, 0.817664, 1.887232, 1.156896])),
 (np.float64(99.66580478290912),
  array([1.055392 , 0.8053696, 0.8282624, 1.8795712, 1.2510336])),
 (np.float64(99.66580478290912),
  array([1.055392 , 0.8053696, 0.8282624, 1.8795712, 1.2510336])),
 (np.float64(25.131492308314236),
  array([0.88727808, 0.8273879 , 0.79674778, 2.04195949, 1.21447526])),
 (np.float64(25.131492308314236),
  array([0.88727808, 0.8273879 , 0.79674778, 2.04195949, 1.21447526])),
 (np.float64(3.969389554803653),
  array([0.83848993, 0.87254034, 0.82868361, 1.9032563 , 1.19128679])),
 (np.float64(0.7926949324388347),
  array([0.73791791, 0.87035807, 0.83942746, 1.97943402, 1.28414711])),
 (np.float64(0.7926949324388347),
  array([0.73

In [27]:
res2.x

array([1.04756723, 1.02350737, 0.72660862, 1.37625267, 0.84062289])

In [ ]:
historic

In [1]:
import numpy as np
csv_file = "test.csv"

historic = [(1, 32131, [4, 5, 3]), (2, 65431, [6, 4, 2]), (3, 123, [1, 1, 1])]

# np.asarray(historic)

In [2]:
np.savetxt(csv_file, historic, delimiter=',', fmt="%s")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 3) + inhomogeneous part.

In [3]:
import csv

with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(historic)

In [4]:
res, historic = None

TypeError: cannot unpack non-iterable NoneType object

In [5]:
str(historic)

'[(1, 32131, [4, 5, 3]), (2, 65431, [6, 4, 2]), (3, 123, [1, 1, 1])]'

In [8]:
for i in range(len(historic)):
    line = str(int(historic[i][0]))
    print(line)

1
2
3


In [11]:
import numpy as np

def add_mask(sparam, orientation, value, flow, fhigh, weight):
    """
    Defines an optimization mask dictionary in a standard format
    :param sparam: S-parameter to optimize: S11, S12, S21, S22. String.
    :param orientation: orientation: greater than, less than ('<','>')
    :param value: value to compare to in the opt, in dB: e.g.: S11 < -20 (dB)
    :param freq_lim_low: Lower frequency limit of the window, in GHz
    :param freq_lim_high: Upper frequency limit of the window, in GHz
    :param weight: Weight in the optimization algorithm. Scalar.
    :return a standardized dict with the above parameters
    """
    
    return {'sparam': sparam,  # parameter to optimize
            'orientation': orientation,  # orientation: greater than, less than ('<','>')
            'value': value,  # value to compare to in the opt.: e.g.: S11 < -20 (dB)
            'flow': flow,  # Lower frequency limit, in GHz
            'fhigh': fhigh,  # Upper frequency limit, in GHz
            'weight': weight}  # Weight in the optimization algorithm


def check_masks(masks, sweep_config):
    """
    Checks mask in a masks list
    :param masks: optimization mask (list of dicts, format: see addwindow() func.)
    :param mat: s2p data matrix,
    """

    # Unpack sweep config for convenience
    f_min = int(sweep_config["f_min"] * 1e-9)  # Hz to GHz to compare with masks
    f_max = int(sweep_config["f_max"] * 1e-9)

    print(f_min)
    print(f_max)

    # Define valid parameters
    valid_sparam = ("S11", "S12", "S21", "S22")
    valid_orientation = ("<", ">")

    # Check masks
    for mask in masks:
        
        # unpack mask
        flow = mask['flow']
        fhigh = mask['fhigh']
        sparam = mask['sparam']
        orientation = mask['orientation']
        
        assert flow >= f_min, f"flow cannnot be lower than F_MIN.\nMask: {mask}"
        assert fhigh <= f_max, f"fhigh cannnot be greater than F_MAX.\nMask: {mask}"
        assert flow <= fhigh, f"flow cannnot be greater than fhigh.\nMask: {mask}"
        assert sparam in valid_sparam, f"Unvalid S-param. Allowed: {valid_sparam}\n \
            Mask: {mask}"
        assert orientation in valid_orientation, f"Unvalid orientation. Allowed: \
            {valid_orientation}\nMask: {mask}"

In [12]:

# Set goals
MASKS = [] # List of masks (goal windows)
MASKS.append(add_mask(sparam='S11', orientation='<', value=-20, flow=3, fhigh=3.1, weight=20))
MASKS.append(add_mask(sparam='S21', orientation='>', value=-2, flow=3, fhigh=3.1, weight=20))
MASKS.append(add_mask(sparam='S21', orientation='<', value=-15, flow=1, fhigh=2.5, weight=1))
MASKS.append(add_mask(sparam='S21', orientation='<', value=-15, flow=3.9, fhigh=5.5, weight=1))


In [13]:

F_MIN = 2e9  # [Hz] Default: 2e9 (2 GHz)
F_MAX = 6e9  # [Hz] Default: 6e9 (6 GHz)
N_POINTS = 201  # Number of measurement points. Default: 201

# dictionary that contains the sweep configuration
SWEEP_CONFIG = {
    "f_min": F_MIN,  # minimum frequency
    "f_max": F_MAX,  # maximum frequency
    "n_points": N_POINTS,  # number of points
    "freq": np.linspace(F_MIN, F_MAX, N_POINTS)  # freq vector
}

In [14]:

# Checks if masks are correctly defined
try:
    check_masks(masks=MASKS, sweep_config=SWEEP_CONFIG)  
    print("Masks checked")

except AssertionError as e:
    print(f"MASKS ARE NOT CORRECTLY SETUP:\n{str(e)}")


2
6
MASKS ARE NOT CORRECTLY SETUP:
flow cannnot be lower than F_MIN.
Mask: {'sparam': 'S21', 'orientation': '<', 'value': -15, 'flow': 1, 'fhigh': 2.5, 'weight': 1}


In [21]:
def shutdown():
    print("shutdown")

In [23]:
while True:
    if input("shutdown? [y/n]") in ("y", "Y"):
        shutdown()
        break

shutdown? [y/n] n
shutdown? [y/n] n
shutdown? [y/n] sadasd
shutdown? [y/n] y


shutdown
